In [1]:
from xdsl.dialects.experimental import aie
from xdsl.dialects.builtin import Region, IndexType, ModuleOp, i32, IntegerAttr, ArrayAttr, i64, StringAttr
from xdsl.dialects.arith import Constant
from xdsl.dialects.memref import MemRefType
from xdsl.dialects import builtin, arith, memref, func
from xdsl.builder import Builder
from xdsl.traits import SymbolTable

def I32Attr(value):
    return IntegerAttr.from_int_and_width(value, i32)

In [2]:
def tutorial7():
    tile14 = aie.TileOp(I32Attr(1), I32Attr(4))
    tile34 = aie.TileOp(I32Attr(3), I32Attr(4))
    tile35 = aie.TileOp(I32Attr(3), I32Attr(5))
    
    object_fifo = aie.createObjectFifo(I32Attr(256), tile14, [tile34,tile35], i32, [256], "of")

    lock34_8 = aie.LockOp(I32Attr(8), I32Attr(0), tile34, StringAttr("lock_a34_8"))
    lock35_8 = aie.LockOp(I32Attr(8), I32Attr(0), tile35, StringAttr("lock_a35_8"))

    @Builder.region
    def region0(builder: Builder):
        builder.insert(tile14)
        builder.insert(tile34)
        builder.insert(tile35)
        builder.insert(object_fifo)
        builder.insert(lock34_8)
        builder.insert(lock35_8)

    device = aie.DeviceOp(I32Attr(0), region0)

    # CORE 14
    acquire_fifo = aie.ObjectFifoAcquireOp(I32Attr(aie.PRODUCE_PORT), I32Attr(1), object_fifo.sym_name, device)

    subview = aie.ObjectFIFOSubviewAccessOp(I32Attr(0), acquire_fifo)
    
    val = arith.Constant.from_int_and_width(14, i32)
    idx = arith.Constant.from_int_and_width(3, IndexType())
    memref_store = memref.Store.get(val, subview, idx)

    release_fifo = aie.ObjectFIFOReleaseOp(I32Attr(aie.PRODUCE_PORT), I32Attr(1), object_fifo.sym_name)

    @Builder.region
    def core14_region(builder: Builder):
        builder.insert(acquire_fifo)
        builder.insert(subview)
        builder.insert(val)
        builder.insert(idx)
        builder.insert(memref_store)
        builder.insert(release_fifo)
        builder.insert(aie.EndOp())

    core14 = aie.CoreOp(I32Attr(1), tile14, core14_region)

    # CORE 34
    uselock34_8_acquire = aie.UseLockOp(I32Attr(0), I32Attr(aie.LOCK_ACQUIRE), I32Attr(aie.BLOCKING), lock34_8)
    acquire_fifo_34 = aie.ObjectFifoAcquireOp(I32Attr(aie.CONSUME_PORT), I32Attr(1), object_fifo.sym_name, device)

    subview_34 = aie.ObjectFIFOSubviewAccessOp(I32Attr(0), acquire_fifo_34)
    
    idx1 = arith.Constant.from_int_and_width(3, IndexType())
    d1 = memref.Load.get(subview_34, idx1)
    c1 = arith.Constant.from_int_and_width(100, i32)
    d2 = arith.Addi(d1, c1)
    idx2 = arith.Constant.from_int_and_width(5, IndexType())
    memref_store_d2_in = memref.Store.get(d2, subview_34, idx2)

    release_fifo_34 = aie.ObjectFIFOReleaseOp(I32Attr(aie.CONSUME_PORT), I32Attr(1), object_fifo.sym_name)
    uselock34_8_release = aie.UseLockOp(I32Attr(1), I32Attr(aie.LOCK_RELEASE), I32Attr(aie.BLOCKING), lock34_8)

    @Builder.region
    def core34_region(builder: Builder):
        builder.insert(uselock34_8_acquire)
        builder.insert(acquire_fifo_34)
        builder.insert(subview_34)
        builder.insert(idx1)
        builder.insert(d1)
        builder.insert(c1)
        builder.insert(d2)
        builder.insert(idx2)
        builder.insert(memref_store_d2_in)
        builder.insert(release_fifo_34)
        builder.insert(uselock34_8_release)
        builder.insert(aie.EndOp())

    core34 = aie.CoreOp(I32Attr(1), tile34, core34_region)

    # CORE 35
    uselock35_8_acquire = aie.UseLockOp(I32Attr(0), I32Attr(aie.LOCK_ACQUIRE), I32Attr(aie.BLOCKING), lock35_8)
    acquire_fifo_35 = aie.ObjectFifoAcquireOp(I32Attr(aie.CONSUME_PORT), I32Attr(1), object_fifo.sym_name, device)

    subview_35 = aie.ObjectFIFOSubviewAccessOp(I32Attr(0), acquire_fifo_35)
    
    idx1 = arith.Constant.from_int_and_width(3, IndexType())
    d1 = memref.Load.get(subview_35, idx1)
    c1 = arith.Constant.from_int_and_width(100, i32)
    d2 = arith.Addi(d1, c1)
    idx2 = arith.Constant.from_int_and_width(5, IndexType())
    memref_store_d2_in = memref.Store.get(d2, subview_35, idx2)

    release_fifo_35 = aie.ObjectFIFOReleaseOp(I32Attr(aie.CONSUME_PORT), I32Attr(1), object_fifo.sym_name)
    uselock35_8_release = aie.UseLockOp(I32Attr(1), I32Attr(aie.LOCK_RELEASE), I32Attr(aie.BLOCKING), lock35_8)

    @Builder.region
    def core35_region(builder: Builder):
        builder.insert(uselock35_8_acquire)
        builder.insert(acquire_fifo_35)
        builder.insert(subview_35)
        builder.insert(idx1)
        builder.insert(d1)
        builder.insert(c1)
        builder.insert(d2)
        builder.insert(idx2)
        builder.insert(memref_store_d2_in)
        builder.insert(release_fifo_35)
        builder.insert(uselock35_8_release)
        builder.insert(aie.EndOp())

    core35 = aie.CoreOp(I32Attr(1), tile35, core35_region)

    region0.block.add_op(core14)
    region0.block.add_op(core34)
    region0.block.add_op(core35)

    module = ModuleOp([device])

    print(module)

In [3]:
tutorial7()

builtin.module {
  AIE.device(xcvc1902) {
    %0 = "AIE.tile"() {"col" = 1 : i32, "row" = 4 : i32} : () -> index
    %1 = "AIE.tile"() {"col" = 3 : i32, "row" = 4 : i32} : () -> index
    %2 = "AIE.tile"() {"col" = 3 : i32, "row" = 5 : i32} : () -> index
    AIE.objectFifo @of(%0, {%1, %2}, 256 : i32) : !AIE.objectFifo<memref<256xi32>>
    %3 = "AIE.lock"(%1) {"lockID" = 8 : i32, "init" = 0 : i32, "sym_name" = "lock_a34_8"} : (index) -> index
    %4 = "AIE.lock"(%2) {"lockID" = 8 : i32, "init" = 0 : i32, "sym_name" = "lock_a35_8"} : (index) -> index
    %5 = AIE.core (%0) {
      %6 = AIE.objectFifo.acquire @of (Produce, 1) : !AIE.objectFifoSubview<memref<256xi32>>
      %7 = AIE.objectFifo.subview.access %6[0] : !AIE.objectFifoSubview<memref<256xi32>> -> memref<256xi32>
      %8 = arith.constant 14 : i32
      %9 = arith.constant 3 : index
      memref.store %8, %7[%9] : memref<256xi32>
      AIE.objectFifo.release @of (Produce, 1)
      "AIE.end"() : () -> ()
    }
    %10 = AIE.core